## 103590450 四資四 馬茂源

In [1]:
from pyspark import SparkConf, SparkContext, SQLContext
#from pyspark.sql.functions import mean, max as max_, min as min_, stddev, count
import os

In [2]:
conf = (SparkConf()
        #.setMaster('spark://10.100.5.182:7077')
        #.setMaster("local")
        .setAppName("hw1"))

In [3]:
sc = SparkContext(conf=conf)
sql_sc = SQLContext(sc)

In [4]:
# rdd = (sc.textFile('./household_power_consumption.csv')
#        .map(lambda line: line.split(';')))
# header = rdd.first()
# data = sql_sc.createDataFrame(rdd.filter(lambda line: line != header), header)
# data.show()

In [5]:
# data = pd.read_csv('./household_power_consumption.csv', 
#                    delimiter=';', 
#                    dtype={'Date':str,
#                           'Time':str,
#                           'Global_active_power':str,
#                           'Global_reactive_power':str,
#                           'Voltage':str,
#                           'Global_intensity':str,
#                           'Sub_metering_1':str,
#                           'Sub_metering_2':str,
#                           'sub_metering_3':str,})
# data = sql_sc.createDataFrame(data)

In [6]:
data = sql_sc.read.csv('./household_power_consumption.csv', sep=';', header=True)

In [7]:
data = data.drop('Date', 'Time', 'Sub_metering_1', 
        'Sub_metering_2', 
        'Sub_metering_3')

In [8]:
data.dtypes

[('Global_active_power', 'string'),
 ('Global_reactive_power', 'string'),
 ('Voltage', 'string'),
 ('Global_intensity', 'string')]

In [9]:
data = (data.withColumn('Global_active_power', data.Global_active_power.cast('float'))
        .withColumn('Global_reactive_power', data.Global_reactive_power.cast('float'))
        .withColumn('Voltage', data.Voltage.cast('float'))
        .withColumn('Global_intensity', data.Global_intensity.cast('float')))
data.dtypes

[('Global_active_power', 'float'),
 ('Global_reactive_power', 'float'),
 ('Voltage', 'float'),
 ('Global_intensity', 'float')]

In [10]:
data = data.na.drop()

1. Output the **minimum**, **maximum**, and **count** of the columns:`Global_active_power`, `Global_reactive_power`, `Voltage`, and `Global_intensity`
2. Output the **mean** and **standard deviation** of these columns


In [11]:
features = ['Global_active_power', 'Global_reactive_power', 'Voltage', 'Global_intensity']

In [12]:
# def get_count(df, feature):
#     return df.select(feature).count()
# def get_minmax(df, feature):
#     return (data.select(feature)
#             .rdd
#             .sortBy(lambda x: x, ascending=False))

In [14]:
data.select('Global_active_power').rdd.map(lambda x: x['Global_active_power']).max()

11.121999740600586

In [15]:
summary = {f:{} for f in features}

In [19]:
summary['Global_intensity']['rdd'].max()

48.400001525878906

In [31]:
count = data.count()
for f in features:
    rdd = data.select(f).rdd.map(lambda x: x[f])
    summary[f]['count'] = count
    summary[f]['max'] = rdd.max()
    summary[f]['min'] = rdd.min()
    summary[f]['mean'] = rdd.mean()
    summary[f]['stddev'] = rdd.stdev()

In [32]:
summary

{'Global_active_power': {'count': 2049280,
  'max': 11.121999740600586,
  'mean': 1.0916150366540185,
  'min': 0.07599999755620956,
  'rdd': PythonRDD[30] at RDD at PythonRDD.scala:48,
  'stddev': 1.057293903150742},
 'Global_intensity': {'count': 2049280,
  'max': 48.400001525878906,
  'mean': 4.627759313004143,
  'min': 0.20000000298023224,
  'rdd': PythonRDD[31] at RDD at PythonRDD.scala:48,
  'stddev': 4.444395174601214},
 'Global_reactive_power': {'count': 2049280,
  'max': 1.3899999856948853,
  'mean': 0.12371447652515713,
  'min': 0.0,
  'rdd': PythonRDD[32] at RDD at PythonRDD.scala:48,
  'stddev': 0.11272195208358389},
 'Voltage': {'count': 2049280,
  'max': 254.14999389648438,
  'mean': 240.83985796672332,
  'min': 223.1999969482422,
  'rdd': PythonRDD[33] at RDD at PythonRDD.scala:48,
  'stddev': 3.2399858706875015}}

In [ ]:
#summary = data.select(features).describe()
#summary.show()
# +-------+-------------------+---------------------+------------------+-----------------+
# |summary|Global_active_power|Global_reactive_power|           Voltage| Global_intensity|
# +-------+-------------------+---------------------+------------------+-----------------+
# |  count|            2049280|              2049280|           2049280|          2049280|
# |   mean| 1.0916150366540094|   0.1237144765251571|240.83985796672414|4.627759313004169|
# | stddev| 1.0572941611180013|  0.11272197958641254|  3.23998666120589|4.444396258981289|
# |    min|              0.076|                  0.0|             223.2|              0.2|
# |    max|             11.122|                 1.39|            254.15|             48.4|
# +-------+-------------------+---------------------+------------------+-----------------+

3 . Perform **min-max normalization** on the columns to generate normalized output

In [33]:
for feature in features:
    min = summary[feature]['min']
    max = summary[feature]['max'] 
    (data.withColumn('norm_' + feature, (data[feature]-min)/(max-min))
         .select(feature, 'norm_' + feature)
         .show())

+-------------------+------------------------+
|Global_active_power|norm_Global_active_power|
+-------------------+------------------------+
|              4.216|      0.3747963225473887|
|               5.36|     0.47836323183743595|
|              5.374|     0.47963065346431577|
|              5.388|     0.48089807509119564|
|              3.666|     0.32500452429198895|
|               3.52|       0.311787077990743|
|              3.702|      0.3282636238926484|
|                3.7|      0.3280825759940403|
|              3.668|      0.3251855937747528|
|              3.662|     0.32464240691061697|
|              4.448|      0.3957993896768334|
|              5.412|     0.48307082254773914|
|              5.224|     0.46605106819754205|
|              5.268|     0.47003442414510127|
|              4.054|     0.36013036355234324|
|              3.384|      0.2994749359350049|
|               3.27|     0.28915445026889064|
|               3.43|     0.30363933978117225|
|            

In [34]:
sc.stop()